## Convertible bonds valuation and comparative analysis
This notebook demonstrates how to get convertible bonds via Search and how to use the financial-contracts end point of Instrument Pricing Analytics (IPA) and the Data Libraries to conduct valuation of the bonds. Additionally, we provide comparative analytics for the ingested convertible bond universe .

#### Learn more

To learn more about the Data Library for Python please join the LSEG Developer Community. By [registering](https://developers.lseg.com/iam/register) and [logging](https://developers.lseg.com/content/devportal/en_us/initCookie.html) into the LSEG Developer Community portal you will have free access to a number of learning materials like 
 [Quick Start guides](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding using the API, please post them on 
this [Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The LSEG Developer Community will be happy to help. 


## Imports

In [1]:
import refinitiv.data as rd
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import refinitiv.data.content.ipa.financial_contracts as rdf

from refinitiv.data.content.ipa.financial_contracts import bond
from sklearn.preprocessing import MinMaxScaler
from plotly.subplots import make_subplots
from datetime import datetime

pd.options.display.max_columns = None
pd.set_option("mode.chained_assignment", None)


In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x1692500d0 {name='workspace'}>

## Building Convertible bond universe

Below we use search request with the following parameters:
* asset status - Issued
* issue date - between 2023-01-01 and 2024-08-30
* issue size - greater than 100M
* convertible status - Established
* sub-asset type - convertible

We also remove convertble bonds with missing UnderlyingAssetRIC for subsequent analysis.

In [3]:
convertibles = rd.discovery.search(
	view = rd.discovery.Views.GOV_CORP_INSTRUMENTS,
	top = 100,
	filter = "(IsActive eq true and (SearchAllCategoryv2 eq 'Bonds' and RCSConvertible in ('A:BY' 'A:3S' 'A:H1')) and \
	(RCSConvertible eq 'A:BY' and (IssueDate ge 2023-01-01 and IssueDate le 2024-08-20) and SubAssetTypeDesc xeq 'Convertible' and \
	IssueSizeUSD gt 1000000 and AssetStatus eq 'ISS' and IssueSizeUSD gt 100000000 and \
	ConvIssueStatus eq 'EST' and not(UnderlyingAssetSectorDesc in ('other'))))",
	select = "RIC,UnderlyingAssetRIC,IssuerCommonName,CouponRate,MaturityDate,IssueDate,IssueSizeUSD,UnderlyingAssetSectorDesc",
	order_by = "IssuerCommonName,MaturityDate"
)
convertibles = convertibles.drop_duplicates(subset = 'UnderlyingAssetRIC')
convertibles

,RIC,UnderlyingAssetRIC,IssuerCommonName,CouponRate,MaturityDate,IssueDate,IssueSizeUSD,UnderlyingAssetSectorDesc
0,007973AD2=,AEIS.O,Advanced Energy Industries Inc,2.5,2028-09-15,2023-09-12,575000000,Electronics
1,00922RAC9=,ATSG.O,Air Transport Services Group Inc,3.875,2029-08-15,2023-08-14,400000000,Airline
2,CA01921DAA3=,AAUC.TO,Allied Gold Corp,8.75,2028-09-07,2023-09-07,107279000,Metals/Mining
3,03209RAA1=,AMPH.O,Amphastar Pharmaceuticals Inc,2.0,2029-03-15,2023-09-15,345000000,Pharmaceuticals
4,05368MAA4=,CDMO.O,Avid Bioservices Inc,7.0,2029-03-01,2024-03-12,160000000,Pharmaceuticals
...,...,...,...,...,...,...,...,...
162,974637AE0=,WGO,Winnebago Industries Inc,3.25,2030-01-15,2024-01-23,350000000,Automotive Manufacturer
163,98139AAC9=,WK,Workiva Inc,1.25,2028-08-15,2023-08-17,702000000,Information/Data Technology
164,US259660050=,1128.HK,Wynn Macau Ltd,4.5,2029-03-07,2023-03-07,600000000,Gaming
166,NZ283436535=,XRO.AX,Xero Investments Ltd,1.625,2031-06-12,2024-06-12,925000000,Information/Data Technology


## Convertible bond valuation using IPA

First, we build the list of universe with ***bond.Definition***.

In [4]:
universe = [bond.Definition(instrument_code=holding) for holding in convertibles['RIC']]

Below we make the IPA request. Additional pricing paramaters and output fields can be found in the [documentation](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-convertible-bond-contracts).

In [5]:
valuation_date = datetime.now().strftime("%Y-%m-%d")

valuation_df = rdf.Definitions(

    universe= universe,
    pricing_parameters=bond.PricingParameters(
        market_data_date=valuation_date,
        price_side=bond.PriceSide.BID,
    ),
    fields = [
        'ConversionStyle',
        'ConversionTypeDescription',
        'ConversionPeriodStartDates',
        'ConversionPeriodEndDates',
        'UnderlyingAsset',
        'UnderlyingRIC',
        'UnderlyingCcy',
        'UnderlyingPriceSide',
        'UnderlyingVolatilityType',
        'UnderlyingDividendType',
        'ConversionRatio',
        'ConversionPrices',
        'FairPrice',
        'FairPricePercent',
        'BondFloor',
        'BondFloorPercent',
        'Parity',
        'ParityPercent',
        'CheapnessPercent',
        'UnderlyingPrice',
        'UnderlyingFlatVolatilityTenor',
        'UnderlyingFlatVolatilityPercent',
        'UnderlyingFxVolatilityPercent',
        'UnderlyingDividendYieldPercent',
        'ImpliedVolatilityPercent',
        'ContingentConversionTrigger',
        'ContingentConversionTriggerReference',
        'CDSRecoveryRatePercent',
        'ExpectedLifeYears',
        'BreakEvenTimeYears',
        'ConvertibleParValue',
        'ConvertiblePremium',
        'ConvertiblePremiumPercent',
        'Premium',
        'ImpliedSpreadBp',
        'CreditSpreadType',
        'ProbabilityOfMaturityPercent',
        'ProbabilityOfMaturityConversionPercent',
        'ProbabilityOfCallPercent',
        'ProbabilityOfPutPercent',
        'ProbabilityOfConversionPercent',
        'ProbabilityOfForcedConversionPercent',
        'EffectiveDurationYears',
        'SpreadDurationYears',
        'DeltaPercent',
        'DeltaInDealCcy',
        'Gamma',
        'GammaInDealCcy',
        'Vega',
        'Theta',
        'Rho',
        'Omicron',
        'Chi',
        'Epsilon'
 ]
).get_data().data.df

valuation_df

,ConversionStyle,ConversionTypeDescription,ConversionPeriodStartDates,ConversionPeriodEndDates,UnderlyingAsset,UnderlyingRIC,UnderlyingCcy,UnderlyingPriceSide,UnderlyingVolatilityType,UnderlyingDividendType,ConversionRatio,ConversionPrices,FairPrice,FairPricePercent,BondFloor,BondFloorPercent,Parity,ParityPercent,CheapnessPercent,UnderlyingPrice,UnderlyingFlatVolatilityTenor,UnderlyingFlatVolatilityPercent,UnderlyingFxVolatilityPercent,UnderlyingDividendYieldPercent,ImpliedVolatilityPercent,ContingentConversionTrigger,ContingentConversionTriggerReference,CDSRecoveryRatePercent,ExpectedLifeYears,BreakEvenTimeYears,ConvertibleParValue,ConvertiblePremium,ConvertiblePremiumPercent,Premium,ImpliedSpreadBp,CreditSpreadType,ProbabilityOfMaturityPercent,ProbabilityOfMaturityConversionPercent,ProbabilityOfCallPercent,ProbabilityOfPutPercent,ProbabilityOfConversionPercent,ProbabilityOfForcedConversionPercent,EffectiveDurationYears,SpreadDurationYears,DeltaPercent,DeltaInDealCcy,Gamma,GammaInDealCcy,Vega,Theta,Rho,Omicron,Chi,Epsilon
0,AMR,Convertible into Equity of the Issuer,[2023-09-12T00:00:00Z],[2028-09-13T00:00:00Z],"{'instrumentCode': 'AEIS.OQ', 'assetCurrency':...",AEIS.OQ,USD,Last,Flat,ForecastTable,7.2747,[137.46271323903392],1.085462,108.5462,0.937962,93.7962,71.001072,71.001072,-8.741255,97.6,200D,35.829059,0.0,0.4181,17.847516,130,PCP,40.0,3.839522,15.435207,1000,0.1475,14.75,39.516063,709.094158,TermStructure,71.518809,10.169498,0,0.0,0.0,18.311693,2.452079,1.43558,51.055183,362.497273,0.000039,3.714273,0.504876,0.00102,-2.654182,-1.561495,0.0,-0.002332
1,AMR,Convertible into Equity of the Issuer,[2023-08-14T00:00:00Z],[2029-08-13T00:00:00Z],"{'instrumentCode': 'ATSG.O', 'assetCurrency': ...",ATSG.O,USD,Last,Flat,ForecastTable,31.2864,[31.962769765776823],1.039362,103.9362,0.940474,94.0474,41.861203,41.861203,-18.739958,13.38,200D,50.961106,0.0,0.0,<NA>,130,PCP,35.2,4.827449,16.019363,1000,0.098889,9.8889,101.758654,1287.699367,TermStructure,85.587697,4.087963,0,0.0,0.0,10.32434,3.548702,2.070656,44.707192,187.149686,0.000694,1.242799,0.426191,0.004818,-3.690385,-2.168676,0.0,0.0
2,AMR,Convertible into Equity of the Issuer,[2023-09-07T00:00:00Z],[2028-09-06T00:00:00Z],"{'instrumentCode': 'AAUC.TO', 'assetCurrency':...",AAUC.TO,CAD,Last,Flat,ForecastTable,172.711572,[5.789999999842513],1.16585,116.585,1.13237,113.237,36.767167,36.767167,-14.710297,2.93,200D,53.745244,5.515,0.0,<NA>,<NA>,<NA>,35.2,3.835767,9.122052,1000,0.03347,3.347,170.445094,1179.344921,TermStructure,83.584825,2.484478,0,0.0,0.0,13.930698,2.850302,1.669195,21.487902,79.004927,0.024337,1.518033,0.179051,0.002084,-3.36801,-1.981941,0.220849,0.0
3,AMR,Convertible into Equity of the Issuer,[2023-09-15T00:00:00Z],[2029-03-13T00:00:00Z],"{'instrumentCode': 'AMPH.O', 'assetCurrency': ...",AMPH.O,USD,Last,Flat,ForecastTable,15.8821,[62.963965722417065],1.061201,106.1201,0.871148,87.1148,70.03212,70.03212,-11.182801,44.095,200D,39.636893,0.0,0.0,18.055088,130,PCP,35.2,4.228762,18.044006,1000,0.190054,19.0054,34.585245,1061.820524,TermStructure,70.700871,7.824724,0,0.0,0.0,21.474405,2.550205,1.501362,57.225892,400.765054,0.000157,3.05634,0.501514,0.001802,-2.692978,-1.59339,0.0,0.0
4,AMR,Convertible into Equity of the Issuer,[2024-03-12T00:00:00Z],[2029-02-27T00:00:00Z],"{'instrumentCode': 'CDMO.O', 'assetCurrency': ...",CDMO.O,USD,Last,Flat,ForecastTable,101.125,[9.88875154511743],1.805767,180.5767,1.073363,107.3363,104.462125,104.462125,-25.68698,10.33,200D,88.600587,0.0,0.0,12.121745,130,PCP,35.2,4.561111,10.873512,1000,0.732404,73.2404,28.459956,3758.771839,TermStructure,79.875642,20.124358,0,0.0,0.0,0.0,1.896016,1.132534,86.13407,899.774799,0.001124,1.199632,0.484728,-0.006131,-3.44573,-2.065853,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,AMR,Convertible into Equity of the Iss

The library returns some of the fields in arrays. Below we remove bonds with missing values and extract the elements into respective columns.

In [6]:
cols_with_arrays = ['ConversionPeriodStartDates', 'ConversionPeriodEndDates', 'ConversionPrices']
valuation_df = valuation_df.dropna(subset=cols_with_arrays)
for col in cols_with_arrays:
    valuation_df[col] = valuation_df[col].apply(lambda x: x[0])
valuation_df

,ConversionStyle,ConversionTypeDescription,ConversionPeriodStartDates,ConversionPeriodEndDates,UnderlyingAsset,UnderlyingRIC,UnderlyingCcy,UnderlyingPriceSide,UnderlyingVolatilityType,UnderlyingDividendType,ConversionRatio,ConversionPrices,FairPrice,FairPricePercent,BondFloor,BondFloorPercent,Parity,ParityPercent,CheapnessPercent,UnderlyingPrice,UnderlyingFlatVolatilityTenor,UnderlyingFlatVolatilityPercent,UnderlyingFxVolatilityPercent,UnderlyingDividendYieldPercent,ImpliedVolatilityPercent,ContingentConversionTrigger,ContingentConversionTriggerReference,CDSRecoveryRatePercent,ExpectedLifeYears,BreakEvenTimeYears,ConvertibleParValue,ConvertiblePremium,ConvertiblePremiumPercent,Premium,ImpliedSpreadBp,CreditSpreadType,ProbabilityOfMaturityPercent,ProbabilityOfMaturityConversionPercent,ProbabilityOfCallPercent,ProbabilityOfPutPercent,ProbabilityOfConversionPercent,ProbabilityOfForcedConversionPercent,EffectiveDurationYears,SpreadDurationYears,DeltaPercent,DeltaInDealCcy,Gamma,GammaInDealCcy,Vega,Theta,Rho,Omicron,Chi,Epsilon
0,AMR,Convertible into Equity of the Issuer,2023-09-12T00:00:00Z,2028-09-13T00:00:00Z,"{'instrumentCode': 'AEIS.OQ', 'assetCurrency':...",AEIS.OQ,USD,Last,Flat,ForecastTable,7.2747,137.462713,1.085462,108.5462,0.937962,93.7962,71.001072,71.001072,-8.741255,97.6,200D,35.829059,0.0,0.4181,17.847516,130,PCP,40.0,3.839522,15.435207,1000,0.1475,14.75,39.516063,709.094158,TermStructure,71.518809,10.169498,0,0.0,0.0,18.311693,2.452079,1.43558,51.055183,362.497273,0.000039,3.714273,0.504876,0.00102,-2.654182,-1.561495,0.0,-0.002332
1,AMR,Convertible into Equity of the Issuer,2023-08-14T00:00:00Z,2029-08-13T00:00:00Z,"{'instrumentCode': 'ATSG.O', 'assetCurrency': ...",ATSG.O,USD,Last,Flat,ForecastTable,31.2864,31.962770,1.039362,103.9362,0.940474,94.0474,41.861203,41.861203,-18.739958,13.38,200D,50.961106,0.0,0.0,<NA>,130,PCP,35.2,4.827449,16.019363,1000,0.098889,9.8889,101.758654,1287.699367,TermStructure,85.587697,4.087963,0,0.0,0.0,10.32434,3.548702,2.070656,44.707192,187.149686,0.000694,1.242799,0.426191,0.004818,-3.690385,-2.168676,0.0,0.0
2,AMR,Convertible into Equity of the Issuer,2023-09-07T00:00:00Z,2028-09-06T00:00:00Z,"{'instrumentCode': 'AAUC.TO', 'assetCurrency':...",AAUC.TO,CAD,Last,Flat,ForecastTable,172.711572,5.790000,1.16585,116.585,1.13237,113.237,36.767167,36.767167,-14.710297,2.93,200D,53.745244,5.515,0.0,<NA>,<NA>,<NA>,35.2,3.835767,9.122052,1000,0.03347,3.347,170.445094,1179.344921,TermStructure,83.584825,2.484478,0,0.0,0.0,13.930698,2.850302,1.669195,21.487902,79.004927,0.024337,1.518033,0.179051,0.002084,-3.36801,-1.981941,0.220849,0.0
3,AMR,Convertible into Equity of the Issuer,2023-09-15T00:00:00Z,2029-03-13T00:00:00Z,"{'instrumentCode': 'AMPH.O', 'assetCurrency': ...",AMPH.O,USD,Last,Flat,ForecastTable,15.8821,62.963966,1.061201,106.1201,0.871148,87.1148,70.03212,70.03212,-11.182801,44.095,200D,39.636893,0.0,0.0,18.055088,130,PCP,35.2,4.228762,18.044006,1000,0.190054,19.0054,34.585245,1061.820524,TermStructure,70.700871,7.824724,0,0.0,0.0,21.474405,2.550205,1.501362,57.225892,400.765054,0.000157,3.05634,0.501514,0.001802,-2.692978,-1.59339,0.0,0.0
4,AMR,Convertible into Equity of the Issuer,2024-03-12T00:00:00Z,2029-02-27T00:00:00Z,"{'instrumentCode': 'CDMO.O', 'assetCurrency': ...",CDMO.O,USD,Last,Flat,ForecastTable,101.125,9.888752,1.805767,180.5767,1.073363,107.3363,104.462125,104.462125,-25.68698,10.33,200D,88.600587,0.0,0.0,12.121745,130,PCP,35.2,4.561111,10.873512,1000,0.732404,73.2404,28.459956,3758.771839,TermStructure,79.875642,20.124358,0,0.0,0.0,0.0,1.896016,1.132534,86.13407,899.774799,0.001124,1.199632,0.484728,-0.006131,-3.44573,-2.065853,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,AMR,Convertible into Equity of the Issuer,2024-01-23T00:00:00Z,2030-01-11T00:00:00Z,"{'instrumentCode': 'WGO.N'

Below we request additional fields to proceed with the analysis and merge with our valuation dataframe.

In [7]:
bond_data  = rd.get_data(convertibles['RIC'].to_list(), 
                         fields=['TR.UnderlRIC', 
                                 'TR.YIELDTOMATURITY',
                                 'TR.CREDITSPREAD', 
                                 'TR.FiOptionAdjustedDuration', 
                                 'TR.FiIssuerSPLongRating', 
                                 'TR.FiIssuerCountryName'])
bond_data

,Instrument,Underlying RIC,Yield to Maturity,Credit Spread,Effective Duration,Issuer SP Long Rating,Issuer Country Name
0,007973AD2=,AEIS.O,2.646,170,3.832875,NR,United States
1,00922RAC9=,ATSG.O,7.562,450,4.30173,BB+,United States
2,CA01921DAA3=,AAUC.TO,8.764,<NA>,2.875438,,Canada
3,03209RAA1=,AMPH.O,3.266,250,4.304686,,United States
4,05368MAA4=,CDMO.O,-0.482,<NA>,4.00116,,United States
...,...,...,...,...,...,...,...
129,974637AE0=,WGO,3.829,200,4.912269,BB-,United States
130,98139AAC9=,WK,3.714,210,3.828154,,United States
131,US259660050=,1128.HK,4.993,275,2.68139,BB-,Macao
132,NZ283436535=,XRO.AX,1.394,400,5.609498,,New Zealand


In [8]:
dataset = pd.merge(valuation_df, bond_data, left_on='UnderlyingRIC', right_on='Underlying RIC', how='inner')
dataset

,ConversionStyle,ConversionTypeDescription,ConversionPeriodStartDates,ConversionPeriodEndDates,UnderlyingAsset,UnderlyingRIC,UnderlyingCcy,UnderlyingPriceSide,UnderlyingVolatilityType,UnderlyingDividendType,ConversionRatio,ConversionPrices,FairPrice,FairPricePercent,BondFloor,BondFloorPercent,Parity,ParityPercent,CheapnessPercent,UnderlyingPrice,UnderlyingFlatVolatilityTenor,UnderlyingFlatVolatilityPercent,UnderlyingFxVolatilityPercent,UnderlyingDividendYieldPercent,ImpliedVolatilityPercent,ContingentConversionTrigger,ContingentConversionTriggerReference,CDSRecoveryRatePercent,ExpectedLifeYears,BreakEvenTimeYears,ConvertibleParValue,ConvertiblePremium,ConvertiblePremiumPercent,Premium,ImpliedSpreadBp,CreditSpreadType,ProbabilityOfMaturityPercent,ProbabilityOfMaturityConversionPercent,ProbabilityOfCallPercent,ProbabilityOfPutPercent,ProbabilityOfConversionPercent,ProbabilityOfForcedConversionPercent,EffectiveDurationYears,SpreadDurationYears,DeltaPercent,DeltaInDealCcy,Gamma,GammaInDealCcy,Vega,Theta,Rho,Omicron,Chi,Epsilon,Instrument,Underlying RIC,Yield to Maturity,Credit Spread,Effective Duration,Issuer SP Long Rating,Issuer Country Name
0,AMR,Convertible into Equity of the Issuer,2023-08-14T00:00:00Z,2029-08-13T00:00:00Z,"{'instrumentCode': 'ATSG.O', 'assetCurrency': ...",ATSG.O,USD,Last,Flat,ForecastTable,31.2864,31.962770,1.039362,103.9362,0.940474,94.0474,41.861203,41.861203,-18.739958,13.38,200D,50.961106,0.0,0.0,<NA>,130,PCP,35.2,4.827449,16.019363,1000,0.098889,9.8889,101.758654,1287.699367,TermStructure,85.587697,4.087963,0,0.0,0.0,10.32434,3.548702,2.070656,44.707192,187.149686,0.000694,1.242799,0.426191,0.004818,-3.690385,-2.168676,0.0,0.0,00922RAC9=,ATSG.O,7.562,450,4.30173,BB+,United States
1,AMR,Convertible into Equity of the Issuer,2023-09-07T00:00:00Z,2028-09-06T00:00:00Z,"{'instrumentCode': 'AAUC.TO', 'assetCurrency':...",AAUC.TO,CAD,Last,Flat,ForecastTable,172.711572,5.790000,1.16585,116.585,1.13237,113.237,36.767167,36.767167,-14.710297,2.93,200D,53.745244,5.515,0.0,<NA>,<NA>,<NA>,35.2,3.835767,9.122052,1000,0.03347,3.347,170.445094,1179.344921,TermStructure,83.584825,2.484478,0,0.0,0.0,13.930698,2.850302,1.669195,21.487902,79.004927,0.024337,1.518033,0.179051,0.002084,-3.36801,-1.981941,0.220849,0.0,CA01921DAA3=,AAUC.TO,8.764,<NA>,2.875438,,Canada
2,AMR,Convertible into Equity of the Issuer,2023-09-15T00:00:00Z,2029-03-13T00:00:00Z,"{'instrumentCode': 'AMPH.O', 'assetCurrency': ...",AMPH.O,USD,Last,Flat,ForecastTable,15.8821,62.963966,1.061201,106.1201,0.871148,87.1148,70.03212,70.03212,-11.182801,44.095,200D,39.636893,0.0,0.0,18.055088,130,PCP,35.2,4.228762,18.044006,1000,0.190054,19.0054,34.585245,1061.820524,TermStructure,70.700871,7.824724,0,0.0,0.0,21.474405,2.550205,1.501362,57.225892,400.765054,0.000157,3.05634,0.501514,0.001802,-2.692978,-1.59339,0.0,0.0,03209RAA1=,AMPH.O,3.266,250,4.304686,,United States
3,AMR,Convertible into Equity of the Issuer,2024-03-12T00:00:00Z,2029-02-27T00:00:00Z,"{'instrumentCode': 'CDMO.O', 'assetCurrency': ...",CDMO.O,USD,Last,Flat,ForecastTable,101.125,9.888752,1.805767,180.5767,1.073363,107.3363,104.462125,104.462125,-25.68698,10.33,200D,88.600587,0.0,0.0,12.121745,130,PCP,35.2,4.561111,10.873512,1000,0.732404,73.2404,28.459956,3758.771839,TermStructure,79.875642,20.124358,0,0.0,0.0,0.0,1.896016,1.132534,86.13407,899.774799,0.001124,1.199632,0.484728,-0.006131,-3.44573,-2.065853,0.0,0.0,05368MAA4=,CDMO.O,-0.482,<NA>,4.00116,,United States
4,AMR,Exchangeable into Equity of a Different (Other...,2029-02-07T00:00:00Z,2029-02-13T00:00:00Z,"{'instrumentCode': 'MSFT.OQ', 'assetCurrency':...",MSFT.OQ,USD,Last,Flat,ForecastTable,1.888373,529.556301,0.969685,96.9685,0.877979,87.7979,75.61991,75.61991,3.676967,400.45,200D,20.250078,0.0,0.753,25.818901,<NA>,<NA>,40.0,4.519444,25.454832,1000,0.091705,9.1705,32.946468,<NA>,TermStructure,69.157732,30.842268,0,0.0,0.0,0.0,2.757632,1.646226,46.60532,352.429011,0.000004,6.969036,0.638309,0.003861,-2.651642,-1.591236,0.0,-0.007289,067

## Comparative analysis of the bonds

### Descriptive analysis of ingested universe

Below we group our dataset by countries, ranking and currencies and show the composition of the universe.

In [9]:
cols_to_group = ['Issuer Country Name', 'Issuer SP Long Rating', 'UnderlyingCcy']

fig = make_subplots(rows=1, cols=3, specs=[[{'type':'bar'}, {'type':'pie'}, {'type':'pie'}]])

for i, col in enumerate(cols_to_group):
    df = dataset[[col, 'Instrument']].groupby([col]).count()
    if col == 'Issuer Country Name':
        fig.add_trace(
            go.Bar(y=df.index, x=df['Instrument'], 
                   orientation='h', showlegend=False),
            row=1, col=i+1,
        )
    else:
        fig.add_trace(
            go.Pie(labels=df.index, values=df['Instrument'], 
                   textinfo='label+percent', showlegend=False),
            row=1, col=i+1
        )

fig.update_layout(
    width=1200,
    height=600,
    annotations=[
        dict(text='Geographical Composition', x=0.05, xref='paper', y=1.15, yref='paper', showarrow=False, font=dict(size=16)),
        dict(text='Quality Composition', x=0.5, xref='paper', y=1.15, yref='paper', showarrow=False, font=dict(size=16)),
        dict(text='Currency Composition', x=0.95, xref='paper', y=1.15, yref='paper', showarrow=False, font=dict(size=16))

    ]
)

fig.show()

### Ranking the bonds based on certain metrics and custome weights

First, we define key metrics, our custom weights for scoring and initialize the dataframe with selected metrics.

In [10]:
metrics_with_weights = {
    'CheapnessPercent':0.3, 
    'Yield to Maturity':0.2, 
    'Effective Duration':0.2, 
    'Credit Spread':0.15, 
    'BondFloorPercent':0.15
    }
scored_universe = dataset[['Instrument'] + list(metrics_with_weights.keys())].dropna()
scored_universe

,Instrument,CheapnessPercent,Yield to Maturity,Effective Duration,Credit Spread,BondFloorPercent
0,00922RAC9=,-18.739958,7.562,4.30173,450,94.0474
2,03209RAA1=,-11.182801,3.266,4.304686,250,87.1148
5,DEA38229=,-5.248888,1.98,5.086774,150,92.272
6,KY255900382=,-15.084983,0.08,3.472635,325,107.265
7,09228FAL7=,-16.288177,4.917,4.072875,600,91.3562
...,...,...,...,...,...,...
120,958102AS4=,-3.922712,-3.436,4.120941,210,91.9086
121,98139AAC9=,-3.178545,3.714,3.828154,210,86.1831
122,US259660050=,-7.054315,4.993,2.68139,275,97.265
123,NZ283436535=,-7.207992,1.394,5.609498,400,84.156


Below, we normalize the metrics and calculated the aggregated custom score.

In [11]:
scaler = MinMaxScaler()
for col in list(metrics_with_weights.keys()):
    scored_universe[col] = scaler.fit_transform(scored_universe[col].to_numpy().reshape(-1,1))

scored_universe['agg_score'] = sum(scored_universe[col] * weight for col, weight in metrics_with_weights.items())
scored_universe = scored_universe.sort_values(by='agg_score', ascending=False).reset_index(drop=True)
scored_universe

,Instrument,CheapnessPercent,Yield to Maturity,Effective Duration,Credit Spread,BondFloorPercent,agg_score
0,KR260773640=,1.000000,0.255215,0.830521,0.066398,0.486335,0.600057
1,ES259774110=,0.395555,0.502520,0.792121,0.046278,1.000000,0.534536
2,CH3946467=,0.000000,1.000000,0.428010,0.798793,0.684498,0.508096
3,IT263795261=,0.447609,0.470719,0.790657,0.026157,0.681486,0.492704
4,733245AC8=,0.085054,0.967938,0.254873,0.597586,0.867044,0.489773
...,...,...,...,...,...,...,...
112,US257625044=,0.490406,0.361479,0.102943,0.006036,0.564166,0.325536
113,KR264785634=,0.477574,0.469648,0.000000,0.008048,0.575931,0.324799
114,US268883720=,0.447315,0.306702,0.315262,0.008048,0.402694,0.320199
115,89377MAB5=,0.396890,0.077178,0.542896,0.154930,0.331085,0.315984


The subsequent analysis is conducted on the top 10 convertible bonds from our initial universe based on the derived aggregated score.

In [12]:
top_bond_rics = scored_universe['Instrument'][:10].to_list()
top_bonds = dataset[dataset['Instrument'].isin(top_bond_rics)]
top_bonds

,ConversionStyle,ConversionTypeDescription,ConversionPeriodStartDates,ConversionPeriodEndDates,UnderlyingAsset,UnderlyingRIC,UnderlyingCcy,UnderlyingPriceSide,UnderlyingVolatilityType,UnderlyingDividendType,ConversionRatio,ConversionPrices,FairPrice,FairPricePercent,BondFloor,BondFloorPercent,Parity,ParityPercent,CheapnessPercent,UnderlyingPrice,UnderlyingFlatVolatilityTenor,UnderlyingFlatVolatilityPercent,UnderlyingFxVolatilityPercent,UnderlyingDividendYieldPercent,ImpliedVolatilityPercent,ContingentConversionTrigger,ContingentConversionTriggerReference,CDSRecoveryRatePercent,ExpectedLifeYears,BreakEvenTimeYears,ConvertibleParValue,ConvertiblePremium,ConvertiblePremiumPercent,Premium,ImpliedSpreadBp,CreditSpreadType,ProbabilityOfMaturityPercent,ProbabilityOfMaturityConversionPercent,ProbabilityOfCallPercent,ProbabilityOfPutPercent,ProbabilityOfConversionPercent,ProbabilityOfForcedConversionPercent,EffectiveDurationYears,SpreadDurationYears,DeltaPercent,DeltaInDealCcy,Gamma,GammaInDealCcy,Vega,Theta,Rho,Omicron,Chi,Epsilon,Instrument,Underlying RIC,Yield to Maturity,Credit Spread,Effective Duration,Issuer SP Long Rating,Issuer Country Name
10,AMR,Convertible into Equity of the Issuer,2023-08-11T00:00:00Z,2030-08-01T00:00:00Z,"{'instrumentCode': 'CLNX.MC', 'assetCurrency':...",CLNX.MC,EUR,Last,Flat,ForecastTable,1602.022393,62.421100,1.14333,114.333,1.1055,110.55,53.155103,53.155103,-9.52306,33.18,200D,26.657517,0.0,0.1677,<NA>,<NA>,<NA>,<NA>,5.944487,33.779916,100000,0.03782,3.782,94.609725,349.238576,FlatSpread,90.353764,5.283291,0,0.0,0.0,4.362945,5.046857,2.983559,24.20968,12868.680263,0.000224,246.825009,0.409795,-0.008314,-5.631374,-3.361784,0.0,-0.001571,ES259774110=,CLNX.MC,3.645,130,5.408597,BBB-,Spain
23,AMR,Convertible into Equity of the Issuer,2023-10-25T00:00:00Z,2030-08-31T00:00:00Z,"{'instrumentCode': 'ENI.MI', 'assetCurrency': ...",ENI.MI,EUR,Last,Flat,ForecastTable,5697.583655,17.551300,1.06527,106.527,0.99693,99.693,80.962664,80.962664,-3.78965,14.21,200D,17.904587,0.0,6.8233,11.789445,<NA>,<NA>,40.0,6.011738,16.704038,100000,0.06833,6.833,26.589214,235.735158,TermStructure,80.607796,13.522402,0,0.0,0.682256,5.187546,3.933714,2.485502,34.782496,28160.835463,0.003222,650.515407,0.702411,-0.011034,-4.215628,-2.681486,0.0,-0.041677,IT263795261=,ENI.MI,2.427,80,5.400279,A-,Italy
43,AMR,Convertible into Equity of the Issuer,2024-03-29T00:00:00Z,2031-02-28T00:00:00Z,"{'instrumentCode': '4062.T', 'assetCurrency': ...",4062.T,JPY,Last,Flat,ForecastTable,1113.213848,8983.000000,1.08442,108.442,0.92574,92.574,50.02783,50.02783,-9.398573,4494.0,200D,50.154713,0.0,0.89,29.7643,150,PCP,35.0,6.579813,-223.249042,10000000,0.15867,15.867,96.390688,429.703531,TermStructure,86.783066,0.834332,0,0.0,0.015367,12.367235,4.953204,2.926933,55.829773,2793042.403371,0.0,15360.36388,0.505434,-0.003759,-5.244334,-3.128808,0.0,-0.002672,JP277431122=,4062.T,0.019,80,6.589836,NR,Japan
56,AMR,Convertible into Equity of the Issuer,2023-06-27T00:00:00Z,2029-05-03T00:00:00Z,"{'instrumentCode': 'MBTN.S', 'assetCurrency': ...",MBTN.S,CHF,Last,Flat,ForecastTable,565.626056,176.795250,1.012596,101.259581,0.997957,99.79567,2.901391,2.901391,-53.090859,4.83,200D,142.285723,6.5425,0.0,<NA>,<NA>,<NA>,35.2,4.75678,26.228851,100000,0.014639,1.463911,1537.145775,3921.786453,TermStructure,99.269819,0.15672,0,0.0,0.000118,0.573342,4.209443,2.479112,61.574463,1786.515923,0.000127,3.147466,0.020052,-0.006061,-4.212285,-2.501561,0.016556,0.0,CH3946467=,MBTN.S,22.699,2000,3.339601,,Germany
61,AMR,Convertible into Equity of the Issuer,2024-03-08T00:00:00Z,2030-03-13T00:00:00Z,"{'instrumentCode': 'MSTR.O', 'assetCurrency': ...",MSTR.O,USD,Last,Flat,ForecastTable,0.6677,1497.678598,0.831705,83.1705,0.800164,80.0164,8.710814,8.710814,27.682291,130.46,200D,102.246324,0.0,0.0,<NA>,130,PCP,35.05,4.134304,119.135419,1000,0.031541,3.1541,1119.105328,<NA>,TermStructure,2.707714,0.204458,0,94.97418,0.0,2.113647,3.801496,2.23285,51.114763,44.525121,0.000001,0.145969,0.063

### Conversion analysis

#### Conversion period

In [14]:
fig = px.timeline(top_bonds, x_start='ConversionPeriodStartDates', x_end='ConversionPeriodEndDates', y='UnderlyingRIC',
                  title='Conversion Period: Start and End Dates')

fig.update_yaxes(title='Bond')
fig.update_xaxes(title='Conversion Period')

fig.show()

#### Conversion probabilities

In [37]:
proba_cols = ['ProbabilityOfConversionPercent','ProbabilityOfMaturityPercent', 'ProbabilityOfMaturityConversionPercent', 
             'ProbabilityOfCallPercent', 'ProbabilityOfPutPercent']

fig = go.Figure()

for col in proba_cols:
    fig.add_trace(go.Bar(
        x=top_bonds['Instrument'],
        y=top_bonds[col],
        name=col,
    ))

fig.update_layout(
    title='Probabilities for Each Bond',
    xaxis_title='Bond',
    yaxis_title='Probability (%)',
    barmode='stack'
)

fig.show()


### Price Analysis

In [30]:
price_cols = ['CheapnessPercent', 'ConvertiblePremiumPercent', 'FairPrice', 'BondFloor']


fig = make_subplots(rows=2, cols=1, subplot_titles=("Metrics (Fair Price and Bond Floor)", "Metrics (Prmium and Cheapness)"))

for col in price_cols:
    if col in ['CheapnessPercent', 'ConvertiblePremiumPercent']:
        fig.add_trace(go.Bar(
            x=top_bonds['UnderlyingRIC'],
            y=top_bonds[col],
            name=col,            
        ), row=1, col=1)
    else:

        fig.add_trace(go.Bar(
            x=top_bonds['UnderlyingRIC'],
            y=top_bonds[col],
            name=col,            
        ), row=2, col=1)

fig.update_layout(
    title_text="Options price Analysis Across Bonds",
    barmode='group',
    height=800
)
fig.show()


### Greeks analysis

In [38]:
top_bonds['Delta'] = top_bonds['DeltaPercent']/100
greek_cols =  ['Delta', 'Gamma', 'Vega', 'Theta', 'Rho']

fig = make_subplots(rows=2, cols=1, subplot_titles=("Metrics (Delta, Vega, Rho)", "Metrics (Gamma, Theta)"))

for col in greek_cols:
    if col not in ['Gamma', 'Theta']:
        fig.add_trace(
            go.Bar(x=top_bonds['Instrument'], y=top_bonds[col], name=col), 
                   row=1, col=1
                )
    else:
        fig.add_trace(
            go.Bar(x=top_bonds['Instrument'], y=top_bonds[col], name=col), 
                   row=2, col=1
        )

fig.update_layout(
    title_text="Options Greeks Analysis Across Bonds",
    barmode='group',
    height=800
)
fig.show()


### Pairwise correlation analysis

In [39]:
cor_metrics =  ['BondFloorPercent', 'FairPricePercent', 'CheapnessPercent', 'ConvertiblePremiumPercent']

fig = px.scatter_matrix(top_bonds[['UnderlyingRIC'] + cor_metrics],
    dimensions=cor_metrics,
    title='Pairwise Scatter Matrix of Convertible Bond Metrics',
    labels={col:col.replace('_', ' ') for col in top_bonds.columns}, 
    hover_name="UnderlyingRIC")

fig.update_layout(height=900) 
fig.show()


###  Metrics-wise comparative analysis

In [46]:
radar_metrics =  ['FairPricePercent', 'BondFloorPercent', 'ParityPercent', 
                    'UnderlyingFlatVolatilityPercent', 'ProbabilityOfMaturityPercent']
min_range = top_bonds[radar_metrics].min(axis=1, numeric_only= True).min()
max_range = top_bonds[radar_metrics].max(axis=1, numeric_only= True).max()

fig = go.Figure()

for bond in top_bonds["Instrument"]:
    fig.add_trace(go.Scatterpolar(
        r=top_bonds[top_bonds["Instrument"] == bond][radar_metrics].values[0],
        theta=radar_metrics,
        fill='toself',
        name=bond
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[min_range, max_range]
        )),
    height = 600,
    title="Radar Chart: Comparing Bonds on Various Metrics"
)

fig.show()


In [48]:
rd.close_session()